In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
page = requests.get("https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=2021")
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
t1 = soup.find(class_="t-chart")

In [4]:
t2 = t1.find_all(class_="oddrow")

In [5]:
t3 = t1.select(".oddrow .text_view_data")

In [7]:
valueList = [val.get_text() for val in t3]

In [8]:
valueList

['01/04/21',
 '0.09',
 '0.09',
 '0.09',
 '0.09',
 '0.10',
 '0.11',
 '0.16',
 '0.36',
 '0.64',
 '0.93',
 '1.46',
 '1.66',
 '01/06/21',
 '0.09',
 '0.09',
 '0.09',
 '0.09',
 '0.11',
 '0.14',
 '0.20',
 '0.43',
 '0.74',
 '1.04',
 '1.60',
 '1.81',
 '01/08/21',
 '0.08',
 '0.08',
 '0.08',
 '0.09',
 '0.10',
 '0.14',
 '0.24',
 '0.49',
 '0.81',
 '1.13',
 '1.67',
 '1.87',
 '01/12/21',
 '0.09',
 '0.08',
 '0.09',
 '0.09',
 '0.11',
 '0.14',
 '0.23',
 '0.50',
 '0.83',
 '1.15',
 '1.68',
 '1.88',
 '01/14/21',
 '0.09',
 '0.09',
 '0.09',
 '0.09',
 '0.10',
 '0.16',
 '0.23',
 '0.49',
 '0.82',
 '1.15',
 '1.69',
 '1.88',
 '01/19/21',
 '0.07',
 '0.09',
 '0.09',
 '0.11',
 '0.10',
 '0.14',
 '0.21',
 '0.45',
 '0.78',
 '1.10',
 '1.65',
 '1.84',
 '01/21/21',
 '0.07',
 '0.09',
 '0.09',
 '0.09',
 '0.10',
 '0.13',
 '0.19',
 '0.45',
 '0.79',
 '1.12',
 '1.68',
 '1.87',
 '01/25/21',
 '0.07',
 '0.08',
 '0.09',
 '0.09',
 '0.10',
 '0.13',
 '0.17',
 '0.42',
 '0.73',
 '1.05',
 '1.61',
 '1.80',
 '01/27/21',
 '0.05',
 '0.06',
 